In [1]:
import pandas as pd
import numpy as np

import get_flight_data as gdf # our own repo for outsourced functions
from geopy.distance import geodesic 

list with subdomains of all countries in EU

In [2]:
countries_sub_url = (["avia_par_be.tsv.gz",
              "avia_par_bg.tsv.gz",
              "avia_par_cz.tsv.gz",
              "avia_par_dk.tsv.gz",
              "avia_par_de.tsv.gz",
              "avia_par_ee.tsv.gz",
              "avia_par_ie.tsv.gz",
              "avia_par_el.tsv.gz",
              "avia_par_es.tsv.gz",
              "avia_par_fr.tsv.gz",
              "avia_par_hr.tsv.gz",
              "avia_par_it.tsv.gz",
              "avia_par_cy.tsv.gz",
              "avia_par_lv.tsv.gz",
              "avia_par_lt.tsv.gz",
              "avia_par_lu.tsv.gz",
              "avia_par_hu.tsv.gz",
              "avia_par_mt.tsv.gz",
              "avia_par_nl.tsv.gz",
              "avia_par_at.tsv.gz",
              "avia_par_pl.tsv.gz",
              "avia_par_pt.tsv.gz",
              "avia_par_ro.tsv.gz",
              "avia_par_si.tsv.gz",
              "avia_par_sk.tsv.gz",
              "avia_par_fi.tsv.gz",
              "avia_par_se.tsv.gz",
              "avia_par_is.tsv.gz",
              "avia_par_no.tsv.gz",
              "avia_par_ch.tsv.gz",
              "avia_par_me.tsv.gz",
              "avia_par_mk.tsv.gz"
              "avia_par_tr.tsv.gz"])

# does not contain uk

In [3]:
# example: Belgium
flights_germany  = gdf.load_and_transform_data(countries_sub_url[4])

In [4]:
flights_germany

,route,fr_country,fr_airport,to_country,to_airport,month,flight_d,seat_d,passenger_d
0,DE_EDDB_BE_EBBR,DE,EDDB,BE,EBBR,2019-11-01,47.0,8166.0,7306.0
1,DE_EDDB_BG_LBSF,DE,EDDB,BG,LBSF,2019-11-01,26.0,4914.0,4406.0
2,DE_EDDB_CH_LSGG,DE,EDDB,CH,LSGG,2019-11-01,50.0,7896.0,6586.0
3,DE_EDDB_CH_LSZM,DE,EDDB,CH,LSZM,2019-11-01,45.0,7372.0,6341.0
4,DE_EDDB_DE_EDDK,DE,EDDB,DE,EDDK,2019-11-01,8.0,64.0,14.0
...,...,...,...,...,...,...,...,...,...
161006,DE_EDMA_DE_EDDK,DE,EDMA,DE,EDDK,2002-01-01,0.0,3087.0,1260.0
161007,DE_EDMA_DE_EDDL,DE,EDMA,DE,EDDL,2002-01-01,0.0,4350.0,1910.0
161008,DE_EDNY_DE_EDDF,DE,EDNY,DE,EDDF,2002-01-01,0.0,6940.0,3734.0
161009,DE_EDNY_DE_EDDH,DE,EDNY,DE,EDDH,2002-01-01,0.0,2300.0,923.0


In [5]:
## save
flights_germany.to_csv("../data/flight_data_de.csv",index = False)

## Adding distances

In [6]:
def get_distance(icao1, icao2):

    try:
        coord1 = (df_geo.loc[df_geo['icao'] == icao1, 'latitude'].iloc[0],
                  df_geo.loc[df_geo['icao'] == icao1, 'longitude'].iloc[0])
        try:
            coord2 = (df_geo.loc[df_geo['icao'] == icao2, 'latitude'].iloc[0],
                      df_geo.loc[df_geo['icao'] == icao2, 'longitude'].iloc[0])
            dist = geodesic(coord1, coord2).km
        except:
            dist = np.nan
            missing_ports.append(icao2) ## global    
    except:
        dist = np.nan
        missing_ports.append(icao1)

    return dist

In [7]:
def add_dist_to_country_df(df, df_geo):
    # look for unique connections
    df_unique_conn = df.drop_duplicates(subset='route').copy()
    # add distance column
    df_unique_conn['distance'] = df_unique_conn.apply(lambda x: get_distance(x['fr_airport'], x['to_airport']), axis=1)
    # merge distance on all connections
    return df.merge(df_unique_conn[['route', 'distance']], how='left', on='route')

In [3]:
path_to_data = "../data/"
df_geo = pd.read_csv(path_to_data + "world_airports.csv")
missing_ports = []

In [8]:
flights_germany = add_dist_to_country_df(flights_germany, df_geo)

In [10]:
flights_germany.to_csv("../data/flight_data_de_distance.csv",index = False)